In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString
import re
import geopandas as gpd
from shapely.geometry import Point
from shapely.strtree import STRtree
from collections import defaultdict
import geopandas as gpd
from shapely import shortest_line
# Load your lines


gas  = gpd.read_file(r"C:\Users\rigli\Documents\lk_gas.gpkg")


In [2]:
output = gpd.GeoDataFrame(columns=["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "ID","geometry","distance"])

In [3]:
gas = gas.drop(columns=['fileid', 'obj_id','medium', 'datenherr','datenlieferant','letzte_aenderung','lagebestimmung','breite','objektart','letzte_lieferung'])

In [4]:
adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")

gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")

proj_gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_PROJ_F")

liegeschaften = gpd.read_file("C:\dev\ews\AV_MOpublic-_Liegenschaften_-OGD\AV_MOpublic-_Liegenschaften_-OGD.gpkg", layer="AVZH_LIEGENSCHAFTEN_F")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rigli\AppData\Local\Temp\ipykernel_7536\525989751.py:1: SyntaxWarning: invalid escape sequence '\d'
  adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")
C:\Users\rigli\AppData\Local\Temp\ipykernel_7536\525989751.py:3: SyntaxWarning: invalid escape sequence '\d'
  gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")
C:\Users\rigli\AppData\Local\Temp\ipykernel_7536\525989751.py:5: SyntaxWarning: invalid escape sequence '\d'
  proj_gebäude = gpd.read_file("C:\dev\ews\

In [5]:
#remove useless attributes
liegeschaften.drop(columns=['OBJID','NBIDENT','NUMMER','VOLLSTAENDIGKEIT','FLAECHENMASS','BFSNR','BEARBEITUNGSDATUM','SHAPE.AREA','SHAPE.LEN'], inplace=True)


#filter only gebäude, remove useless attributes
gebäude = gebäude[(gebäude['ART'] == 'Gebäude')]
proj_gebäude = proj_gebäude[proj_gebäude['ARTZHID'].isin([0, 1, 2, 3, 4, 5, 6, 7])]

gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN'], inplace=True)
proj_gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN','BEWILLIGUNGSSTATUS'], inplace=True)

In [6]:
gebäude['Projektiert'] = 'Nein'

In [7]:
proj_gebäude['Projektiert'] = 'Ja'

In [ ]:
all_gebäude = pd.concat([gebäude, proj_gebäude])

all_gebäude =gpd.GeoDataFrame(all_gebäude, geometry='geometry')


In [9]:

# Create GeoDataFrame for first vertices
first_vertices = gpd.GeoDataFrame(
    gas.drop(columns='geometry'),  # keep other columns if needed
    geometry=gas.geometry.apply(lambda line: Point(line.coords[0])),
    crs=gas.crs
)

# Create GeoDataFrame for last vertices
last_vertices = gpd.GeoDataFrame(
    gas.drop(columns='geometry'),  # keep other columns if needed
    geometry=gas.geometry.apply(lambda line: Point(line.coords[-1])),
    crs=gas.crs
)


# Combine into one GeoDataFrame if desired
all_vertices = gpd.GeoDataFrame(
    pd.concat([first_vertices, last_vertices], ignore_index=True),
    crs=gas.crs
)



In [10]:
# Drop duplicate geometries
unique_vertices = all_vertices.drop_duplicates(subset='geometry')

# Display result
print(unique_vertices)

                       tid    eigentuemer          status  \
0       1_ch19r4uf8UuB0loJ      unbekannt      in_Betrieb   
1       1_ch19r4ufli3jbJZW      unbekannt      in_Betrieb   
2       1_ch19r4ufoRvUBy3K      unbekannt      in_Betrieb   
3       1_ch19r4ufiocXxB7b      unbekannt      in_Betrieb   
4       1_ch19r4ufkdLRCz9l      unbekannt      in_Betrieb   
...                    ...            ...             ...   
409308    ch156ym408005f7P       SWISSGAS      in_Betrieb   
409310    ch156ym40800Dwv4       SWISSGAS  ausser_Betrieb   
409313    ch156ym40800Dx2Z  nicht-erfasst      in_Betrieb   
409314    ch156ym40800Dx2H  nicht-erfasst      in_Betrieb   
409315    ch156ym40800Dx2z  nicht-erfasst      in_Betrieb   

                               geometry  
0       POINT (2677239.414 1245963.839)  
1        POINT (2677246.32 1245954.909)  
2       POINT (2677277.831 1245914.189)  
3       POINT (2677285.093 1245904.783)  
4       POINT (2677259.659 1246011.744)  
...            

In [11]:
points = unique_vertices

# Buffer points by 5 cm (0.05 meters)
points['buffered'] = points.geometry.buffer(0.05)

# Create a GeoDataFrame from buffered geometries for spatial join
buffer_gdf = gpd.GeoDataFrame(points.drop(columns='geometry'), geometry=points['buffered'], crs=points.crs)

# Spatial join: find which buffered points intersect which lines
join = gpd.sjoin(buffer_gdf, gas, how='left', predicate='intersects')

# Count number of unique lines each buffered point intersects
counts = join.groupby(join.index).size()

# Filter points that intersect less than 2 lines (keep those)
keep_points = counts[counts < 2].index

# Keep original points corresponding to these indices
anschlüsse = points.loc[keep_points]

print(f"Original points count: {len(points)}")
print(f"Filtered points count (intersect < 2 lines): {len(anschlüsse)}")

c:\dev\ews\.venv\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Original points count: 250031
Filtered points count (intersect < 2 lines): 74791


In [ ]:
# Drop the extra geometry column ('buffered') before saving
anschlüsse = anschlüsse.drop(columns='buffered', errors='ignore')
# Make sure the active geometry is 'geometry' column
anschlüsse.set_geometry('geometry', inplace=True)
anschlüsse['ID'] = range(len(anschlüsse))
anschlüsse.to_file("gas_output.gpkg", layer='GASANSCHLUSS_P')

In [13]:
adressen['ADRESSE'] = adressen['STRASSENNAME'] + ' ' + adressen['HAUSNUMMER']
adressen_clean = adressen[adressen['GWR_EGID'].notna()]
adressen_subset = adressen_clean[['ADRESSE','GWR_EGID','PLZ','ORTSCHAFTSNAME']]

In [ ]:
gebäude_adressen = all_gebäude.merge(adressen_subset, how='left', on='GWR_EGID')

In [31]:
def filter_out_number_dot_number(address):
    # Ensure it's a string before applying regex
    address = str(address)
    return not re.search(r'(?:\d+)?\.\d', address)
    
    
valid_gebäude = gebäude_adressen[gebäude_adressen['ADRESSE'].apply(filter_out_number_dot_number)]



valid_gebäude.to_file("gas_output.gpkg", layer='ALLE_VALIDE_GEBÄUDE_F')
liegeschaften.to_file("gas_output.gpkg", layer='LIEGENSCHAFTEN_F')

In [16]:
#spatial join with gebäude
gebäude_anschluss = valid_gebäude.sjoin(anschlüsse, how='inner', predicate='intersects')
# proj_gebäude_sonden = proj_gebäude_sonden[proj_gebäude_sonden['ARTZH'] == "Gebäude Wohnen"]

#define variante
gebäude_anschluss['VARIANTE'] = 'Anschluss in Gebäude'

output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_7536\630904738.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))


In [17]:
gebäude_anschluss.to_file("gas_output.gpkg", layer='ANSCHLUSS_IN_GEBÄUDE_F')

In [18]:
not_geb_joined = anschlüsse[~anschlüsse['ID'].isin(output['ID'])]

In [19]:

# ─────────────────────────────────────────────────────────────
# Step 1: Spatial join to get parcel ID for points and buildings

anschluss_liegenschaft = gpd.sjoin(
    not_geb_joined, liegeschaften, how="left", predicate='within'
)

# Save original geometry and use centroids for spatial join
valid_gebäude['orig_geom'] = valid_gebäude.geometry
valid_gebäude['geometry'] = valid_gebäude['orig_geom'].centroid

gebäude_liegenschaft = gpd.sjoin(
    valid_gebäude, liegeschaften, how="left", predicate='within'
)

# Restore original geometry
gebäude_liegenschaft['geometry'] = gebäude_liegenschaft['orig_geom']
gebäude_liegenschaft = gebäude_liegenschaft.drop(columns=['orig_geom'])


c:\dev\ews\.venv\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [20]:


# Suppose ansch_gdf is your anschluss_liegenschaft GeoDataFrame
# and buildings_gdf is gebäude_liegenschaft

# Optionally, filter only matching parcels first (to reduce candidates)
# For example, do a spatial join or filter buildings by parcel.

def find_best_building_with_line(point_geom, parcel_id, buildings_gdf):
    # Filter buildings in the same parcel (or pre-indexed)
    candidates = buildings_gdf[buildings_gdf['EGRIS_EGRID'] == parcel_id]
    if candidates.empty:
        return None, None  # no match

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        # compute shortest line between point and building
        line = shortest_line(point_geom, b_geom)
        d = line.length
        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# Now apply in a loop (or vectorized) to your anschluss_gdf
results = []
for _, a in anschluss_liegenschaft.iterrows():
    p = a.geometry
    pid = a['EGRIS_EGRID']
    b_row, line = find_best_building_with_line(p, pid, gebäude_liegenschaft)
    results.append({
        'ansch_id': a.name,
        'tid': a['tid'],
        'eigentuemer': a['eigentuemer'],
        'tid': a['status'],
        'ID': a['ID'],
        'GWR_EGID': (b_row['GWR_EGID'] if b_row is not None else None),
        'line': line,
        'distance': line.length if line is not None else None,
        # add other building attributes as needed
    })

# Convert results to a GeoDataFrame if you like
nearest_building_line = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)


In [21]:
nearest_building_line = nearest_building_line.dropna(subset=['GWR_EGID'])


nearest_building_line.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_LIEGENSCHAFT_L')

In [ ]:

nearest_building_line_subset = nearest_building_line.drop(columns=['line','ansch_id'])

nearest_building = valid_gebäude.merge(nearest_building_line_subset, on='GWR_EGID')

nearest_building['VARIANTE'] = 'Nächstes Gebäude in Liegenschaft'

output = gpd.GeoDataFrame(pd.concat([output, nearest_building[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID","distance"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_7536\1334741963.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, nearest_building[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID","distance"]]], ignore_index=True))


In [23]:
# all_building_in_liegenschaft = anschluss_liegenschaft.merge(gebäude_liegenschaft, on='EGRIS_EGRID')

In [24]:
# all_building_in_liegenschaft = all_building_in_liegenschaft.drop(columns=['geometry_x', 'index_right_x', 'index_right_y', 'EGRIS_EGRID'])

In [25]:
# # Set the geometry column
# all_building_in_liegenschaft = all_building_in_liegenschaft.set_geometry('geometry_y')

# # Rename it to 'geometry' if it's not already called that
# all_building_in_liegenschaft = all_building_in_liegenschaft.rename_geometry('geometry')

In [26]:
# all_building_in_liegenschaft.to_file("gas_output.gpkg",layer="ALLE_GEBÄUDE_IN_LIEGENSCHAFT_F")

# all_building_in_liegenschaft['VARIANTE'] = 'Alle Gebäude in der Liegenschaft'

# output = gpd.GeoDataFrame(pd.concat([output, all_building_in_liegenschaft[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))

In [27]:

building_sindex = gebäude_liegenschaft.sindex

def find_best_building_with_line(point_geom, buildings_gdf, sindex, search_radius=100):
    """
    Finds the closest building to a point using the shortest_line.
    Uses spatial index for fast lookup.
    
    Returns:
        - best matching building row (GeoSeries)
        - shortest line (LineString)
    """
    # Bounding box around the point
    bbox = point_geom.buffer(search_radius).bounds
    candidate_idx = list(sindex.intersection(bbox))

    if not candidate_idx:
        return None, None

    candidates = buildings_gdf.iloc[candidate_idx]

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        line = shortest_line(point_geom, b_geom)
        d = line.length

        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# ------------------------------
# Apply to all points in anschluss_liegenschaft
# ------------------------------

results = []

for _, a in anschluss_liegenschaft.iterrows():
    p = a.geometry
    b_row, line = find_best_building_with_line(
        point_geom=p,
        buildings_gdf=gebäude_liegenschaft,
        sindex=building_sindex,
        search_radius=100  # adjust as needed
    )

    results.append({
        'ansch_id': a.name,
        'tid': a.get('tid'),
        'eigentuemer': a.get('eigentuemer'),
        'status': a.get('status'),
        'ID': a.get('ID'),
        'GWR_EGID': b_row.get('GWR_EGID') if b_row is not None else None,
        'line': line,
        'distance': line.length if line is not None else None,
    })

# ------------------------------
# Convert to GeoDataFrame
# ------------------------------

nearest_building_line_all = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)

In [ ]:
nearest_building_line_all = nearest_building_line_all.dropna(subset=['GWR_EGID'])


nearest_building_line_all.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_L')

nearest_building_line_all_subset = nearest_building_line_all.drop(columns=['line','ansch_id'])

nearest_building_all = valid_gebäude.merge(nearest_building_line_all_subset, on='GWR_EGID')

nearest_building_all['VARIANTE'] = 'Nächstes Gebäude'

output = gpd.GeoDataFrame(pd.concat([output, nearest_building_all[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID","distance"]]], ignore_index=True))


nearest_building_all.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_F')

In [29]:
import datetime

today = datetime.date.today()


output['GWR_EGID'] = output['GWR_EGID'].fillna(0).astype(int)
output.to_csv(f"gas_output_{today}.csv")

In [ ]:
output